In [1]:
!pip install earthengine-api geemap shapely geopandas xgboost tensorflow tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.1 MB/s eta 0:00:00


In [3]:
import ee
import os

# Authenticate & Initialize Earth Engine
try:
    ee.Initialize()
    print("✅ Earth Engine already initialized.")
except Exception:
    print("🌍 Authenticating Google Earth Engine...")
    ee.Authenticate()  # Opens a link for Google Sign-in
    ee.Initialize(project='bloomwatch-474009')
    print("✅ Earth Engine initialized successfully.")


🌍 Authenticating Google Earth Engine...
✅ Earth Engine initialized successfully.


In [4]:
import json
import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
from tqdm import tqdm
import geemap
from shapely.geometry import box
import geopandas as gpd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import r2_score, mean_squared_error
import warnings

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [5]:
# Approximate bounding box around Nashik district, Maharashtra, India
lat_min, lon_min, lat_max, lon_max = 19.8, 73.6, 20.2, 74.0
REGION = [lon_min, lat_min, lon_max, lat_max]

START_DATE = '2019-01-01'
END_DATE = datetime.date.today().isoformat()
FORECAST_YEARS = 1
TILE_SIZE_DEG = 0.05
BLOOM_PROB_THRESHOLD = 0.6
BLOOM_MODERATE_LOW = 0.3
OUTPUT_JSON = '/content/bloom_predictions_per_tile.json'


In [6]:
def create_tiles(lon_min, lat_min, lon_max, lat_max, tile_size_deg=TILE_SIZE_DEG):
    tiles = []
    ix = 0
    lon = lon_min
    while lon < lon_max:
        jx = 0
        lat = lat_min
        while lat < lat_max:
            b = box(lon, lat, min(lon+tile_size_deg, lon_max), min(lat+tile_size_deg, lat_max))
            tiles.append({
                'tile_id': f'tile_{ix}_{jx}',
                'geometry': b,
                'lon_min': lon,
                'lat_min': lat,
                'lon_max': min(lon+tile_size_deg, lon_max),
                'lat_max': min(lat+tile_size_deg, lat_max)
            })
            lat += tile_size_deg
            jx += 1
        lon += tile_size_deg
        ix += 1
    return gpd.GeoDataFrame(tiles)

lon_min, lat_min, lon_max, lat_max = REGION
tiles_gdf = create_tiles(lon_min, lat_min, lon_max, lat_max)
print(f"✅ Created {len(tiles_gdf)} tiles for Nashik region.")


✅ Created 72 tiles for Nashik region.


In [7]:
def ee_time_series_to_df(collection_id, band, geom, start_date, end_date, scale=1000):
    try:
        col = ee.ImageCollection(collection_id).select(band).filterDate(start_date, end_date).filterBounds(geom)
        def img_to_feature(img):
            stat = img.reduceRegion(ee.Reducer.mean(), geom, scale=scale)
            return ee.Feature(None, {'date': img.date().format('YYYY-MM-dd'), band: stat.get(band)})
        fc = col.map(img_to_feature)
        info = fc.getInfo()
        if not info or 'features' not in info or len(info['features']) == 0:
            print(f"⚠️ No data for {band} in {collection_id}")
            return pd.DataFrame(columns=['date', band])
        features = info['features']
        rows = [{'date': f['properties']['date'], band: f['properties'].get(band, None)} for f in features]
        df = pd.DataFrame(rows)
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date').dropna()
        return df
    except Exception as e:
        print(f"❌ Error fetching {band} from {collection_id}: {e}")
        return pd.DataFrame(columns=['date', band])


In [8]:
def extract_features_for_tile(tile_row, start_date=START_DATE, end_date=END_DATE, scale=1000):
    geom = ee.Geometry.Rectangle([
        tile_row.lon_min, tile_row.lat_min,
        tile_row.lon_max, tile_row.lat_max
    ])

    dfs = []
    current_start = pd.to_datetime(start_date)
    current_end = pd.to_datetime(end_date)

    # Split into yearly chunks to avoid 5000-element limit
    while current_start < current_end:
        chunk_start = current_start.strftime('%Y-%m-%d')
        chunk_end = (current_start + pd.DateOffset(years=1)).strftime('%Y-%m-%d')
        print(f"🛰️ Fetching data for {chunk_start} → {chunk_end}...")

        try:
            chunk_dfs = []

            # Vegetation indices
            ndvi_df = ee_time_series_to_df('MODIS/061/MOD13A1', 'NDVI', geom, chunk_start, chunk_end, 250)
            ndvi_df = ndvi_df.rename(columns={'NDVI': 'ndvi'})
            chunk_dfs.append(ndvi_df)

            evi_df = ee_time_series_to_df('MODIS/061/MOD13A1', 'EVI', geom, chunk_start, chunk_end, 250)
            evi_df = evi_df.rename(columns={'EVI': 'evi'})
            chunk_dfs.append(evi_df)

            # Land Surface Temperature (Day/Night)
            lst_day_df = ee_time_series_to_df('MODIS/061/MOD11A1', 'LST_Day_1km', geom, chunk_start, chunk_end, 1000)
            lst_day_df = lst_day_df.rename(columns={'LST_Day_1km': 'lst_day'})
            chunk_dfs.append(lst_day_df)

            lst_night_df = ee_time_series_to_df('MODIS/061/MOD11A1', 'LST_Night_1km', geom, chunk_start, chunk_end, 1000)
            lst_night_df = lst_night_df.rename(columns={'LST_Night_1km': 'lst_night'})
            chunk_dfs.append(lst_night_df)

            # Reflectance bands
            for b in ['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04']:
                ref_df = ee_time_series_to_df('MODIS/061/MOD09GA', b, geom, chunk_start, chunk_end, 500)
                ref_df = ref_df.rename(columns={b: f'reflectance_{b}'})
                chunk_dfs.append(ref_df)

            # Precipitation
            precip_df = ee_time_series_to_df('UCSB-CHG/CHIRPS/PENTAD', 'precipitation', geom, chunk_start, chunk_end, 5000)
            precip_df = precip_df.rename(columns={'precipitation': 'precip'})
            chunk_dfs.append(precip_df)

            # Soil moisture (surface + root zone)
            sm_surface_df = ee_time_series_to_df('NASA/GLDAS/V021/NOAH/G025/T3H', 'SoilMoi0_10cm_inst', geom, chunk_start, chunk_end, 25000)
            sm_surface_df = sm_surface_df.rename(columns={'SoilMoi0_10cm_inst': 'soil_moisture_surface'})
            chunk_dfs.append(sm_surface_df)

            sm_root_df = ee_time_series_to_df('NASA/GLDAS/V021/NOAH/G025/T3H', 'SoilMoi10_40cm_inst', geom, chunk_start, chunk_end, 25000)
            sm_root_df = sm_root_df.rename(columns={'SoilMoi10_40cm_inst': 'soil_moisture_root'})
            chunk_dfs.append(sm_root_df)

            # Merge all valid dataframes for this year
            valid_dfs = [d for d in chunk_dfs if not d.empty]
            if valid_dfs:
                df_chunk = valid_dfs[0]
                for d in valid_dfs[1:]:
                    overlapping_cols = [c for c in d.columns if c in df_chunk.columns and c != 'date']
                    if overlapping_cols:
                        d = d.drop(columns=overlapping_cols)
                    df_chunk = pd.merge(df_chunk, d, on='date', how='outer')
                dfs.append(df_chunk)

        except Exception as e:
            print(f"⚠️ Error fetching chunk {chunk_start}-{chunk_end}: {e}")

        current_start += pd.DateOffset(years=1)

    # Merge all yearly chunks safely
    if not dfs:
        print(f"⚠️ No valid data for {tile_row.tile_id}")
        return pd.DataFrame()

    df = dfs[0]
    for d in dfs[1:]:
        overlapping_cols = [c for c in d.columns if c in df.columns and c != 'date']
        if overlapping_cols:
            d = d.drop(columns=overlapping_cols)
        df = pd.merge(df, d, on='date', how='outer')

    # Clean + interpolate missing values
    df = (
        df.sort_values('date')
          .set_index('date')
          .interpolate(method='time')
          .ffill()
          .bfill()
          .dropna()
          .reset_index()
    )

    print(f"✅ Extracted {len(df)} records for {tile_row.tile_id}")
    return df


In [9]:
from tqdm import tqdm
import os

os.makedirs("tiles_data", exist_ok=True)
all_tiles = []

print("🚀 Starting feature extraction for all region tiles...\n")

for idx, row in tqdm(tiles_gdf.iterrows(), total=len(tiles_gdf)):
    print(f"🛰️ Processing tile: {row.tile_id}...")
    df_tile = extract_features_for_tile(row)
    if not df_tile.empty:
        df_tile['tile_id'] = row.tile_id
        all_tiles.append(df_tile)
        df_tile.to_csv(f"tiles_data/{row.tile_id}_timeseries.csv", index=False)
        print(f"✅ Saved: tiles_data/{row.tile_id}_timeseries.csv\n")

# Combine all tiles (if needed)
if all_tiles:
    full_df = pd.concat(all_tiles, ignore_index=True)
    full_df.to_csv("nashik_region_full_timeseries.csv", index=False)
    print("💾 All tiles merged into nashik_region_full_timeseries.csv")
else:
    print("⚠️ No valid data extracted for any tiles.")


🚀 Starting feature extraction for all region tiles...



  0%|          | 0/72 [00:00<?, ?it/s]

🛰️ Processing tile: tile_0_0...
🛰️ Fetching data for 2019-01-01 → 2020-01-01...
🛰️ Fetching data for 2020-01-01 → 2021-01-01...
🛰️ Fetching data for 2021-01-01 → 2022-01-01...
🛰️ Fetching data for 2022-01-01 → 2023-01-01...
🛰️ Fetching data for 2023-01-01 → 2024-01-01...
🛰️ Fetching data for 2024-01-01 → 2025-01-01...
🛰️ Fetching data for 2025-01-01 → 2026-01-01...
✅ Extracted 155844 records for tile_0_0


  1%|▏         | 1/72 [01:37<1:54:57, 97.15s/it]

✅ Saved: tiles_data/tile_0_0_timeseries.csv

🛰️ Processing tile: tile_0_1...
🛰️ Fetching data for 2019-01-01 → 2020-01-01...
🛰️ Fetching data for 2020-01-01 → 2021-01-01...
🛰️ Fetching data for 2021-01-01 → 2022-01-01...
🛰️ Fetching data for 2022-01-01 → 2023-01-01...
🛰️ Fetching data for 2023-01-01 → 2024-01-01...
🛰️ Fetching data for 2024-01-01 → 2025-01-01...
🛰️ Fetching data for 2025-01-01 → 2026-01-01...
✅ Extracted 155844 records for tile_0_1


...

 99%|█████████▊| 71/72 [2:03:43<01:34, 94.15s/it]

⚠️ No valid data for tile_8_6
🛰️ Processing tile: tile_8_7...
🛰️ Fetching data for 2019-01-01 → 2020-01-01...
🛰️ Fetching data for 2020-01-01 → 2021-01-01...
🛰️ Fetching data for 2021-01-01 → 2022-01-01...
🛰️ Fetching data for 2022-01-01 → 2023-01-01...
🛰️ Fetching data for 2023-01-01 → 2024-01-01...
🛰️ Fetching data for 2024-01-01 → 2025-01-01...
🛰️ Fetching data for 2025-01-01 → 2026-01-01...


100%|██████████| 72/72 [2:05:19<00:00, 104.44s/it]

⚠️ No valid data for tile_8_7


💾 All tiles merged into nashik_region_full_timeseries.csv


In [33]:
import glob

print("📦 Combining all extracted tile data...")

# Read all CSVs from the tiles_data folder
csv_files = sorted(glob.glob("tiles_data/tile_*_timeseries.csv"))

if not csv_files:
    raise FileNotFoundError("⚠️ No CSVs found in tiles_data/. Please ensure feature extraction completed successfully.")

# Combine all CSVs into one DataFrame
all_dfs = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        df["tile_id"] = file.split("/")[-1].replace("_timeseries.csv", "")
        all_dfs.append(df)
    except Exception as e:
        print(f"❌ Error reading {file}: {e}")

combined_df = pd.concat(all_dfs, ignore_index=True)
print(f"✅ Combined {len(all_dfs)} tiles with {len(combined_df):,} total records")

# Clean and format
combined_df["date"] = pd.to_datetime(combined_df["date"], errors="coerce")
combined_df = combined_df.dropna(subset=["date"])
combined_df = combined_df.sort_values(["tile_id", "date"]).reset_index(drop=True)

# Basic feature sanity check
print("\n📊 Columns in combined dataset:")
print(combined_df.columns.tolist())
print("\n🧾 Sample data:")
display(combined_df.head())

# Save the merged dataset
combined_df.to_csv("tiles_data/combined_timeseries.csv", index=False)
print("💾 Saved combined dataset as tiles_data/combined_timeseries.csv")


📦 Combining all extracted tile data...
✅ Combined 64 tiles with 9,974,133 total records

📊 Columns in combined dataset:
['date', 'ndvi', 'evi', 'lst_day', 'lst_night', 'reflectance_sur_refl_b01', 'reflectance_sur_refl_b02', 'reflectance_sur_refl_b03', 'reflectance_sur_refl_b04', 'precip', 'soil_moisture_surface', 'soil_moisture_root', 'tile_id']

🧾 Sample data:


,date,ndvi,evi,lst_day,lst_night,reflectance_sur_refl_b01,reflectance_sur_refl_b02,reflectance_sur_refl_b03,reflectance_sur_refl_b04,precip,soil_moisture_surface,soil_moisture_root,tile_id
0,2019-01-01,2511.342953,1319.259377,15050.557511,14336.385413,924.408467,1575.589948,529.708127,849.272694,0.436489,14.454,58.967999,tile_0_0
1,2019-01-01,2511.342953,1319.259377,15050.557511,14336.385413,924.408467,1575.589948,529.708127,849.272694,0.436489,14.391,58.868999,tile_0_0
2,2019-01-01,2511.342953,1319.259377,15050.557511,14336.385413,924.408467,1575.589948,529.708127,849.272694,0.436489,14.391,58.721001,tile_0_0
3,2019-01-01,2511.342953,1319.259377,15050.557511,14336.385413,924.408467,1575.589948,529.708127,849.272694,0.436489,14.391,58.632999,tile_0_0
4,2019-01-01,2511.342953,1319.259377,15050.557511,14336.385413,924.408467,1575.589948,529.708127,849.272694,0.436489,14.391,58.632000,tile_0_0


💾 Saved combined dataset as tiles_data/combined_timeseries.csv


In [34]:
# 🚀 Auto-resumable crop analysis pipeline after tile extraction
import ee, pandas as pd, numpy as np, os, glob

# --- Step 1: Initialize Earth Engine ---
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize(project='cosmic-kite-439012-k4')
# --- Step 2: Define folders ---
data_folder = "tiles_data"
os.makedirs(data_folder, exist_ok=True)

# --- Step 3: Identify already processed tiles ---
processed_tiles = [
    os.path.basename(f).replace("_analysis.csv", "")
    for f in glob.glob(f"{data_folder}/*_analysis.csv")
]
print(f"✅ Found {len(processed_tiles)} processed tiles. Resuming from remaining ones...")

# --- Step 4: Find unprocessed tiles ---
tile_files = [
    f for f in glob.glob(f"{data_folder}/tile_*_timeseries.csv")
    if os.path.basename(f).replace("_timeseries.csv", "") not in processed_tiles
]
print(f"🧩 {len(tile_files)} tiles left to process.")

# --- Step 5: Define analysis function ---
def analyze_crop_health(df, tile_id):
    """
    Performs basic health metrics and saves summary per tile.
    """
    df = df.copy()
    df["ndvi_norm"] = (df["ndvi"] - df["ndvi"].min()) / (df["ndvi"].max() - df["ndvi"].min())
    df["evi_norm"] = (df["evi"] - df["evi"].min()) / (df["evi"].max() - df["evi"].min())
    df["lst_norm"] = (df["lst_day"] - df["lst_day"].min()) / (df["lst_day"].max() - df["lst_day"].min())
    df["moisture_score"] = (df["soil_moisture_surface"] + df["soil_moisture_root"]) / 2

    # Simple health index: NDVI + EVI - Temperature stress
    df["health_index"] = (
        0.4 * df["ndvi_norm"] +
        0.4 * df["evi_norm"] -
        0.2 * df["lst_norm"]
    )

    summary = {
        "tile_id": tile_id,
        "avg_health_index": df["health_index"].mean(),
        "avg_ndvi": df["ndvi"].mean(),
        "avg_evi": df["evi"].mean(),
        "avg_lst": df["lst_day"].mean(),
        "avg_precip": df["precip"].mean(),
        "avg_soil_moisture": df["moisture_score"].mean(),
    }

    pd.DataFrame([summary]).to_csv(f"{data_folder}/{tile_id}_analysis.csv", index=False)
    print(f"✅ Saved: {tile_id}_analysis.csv")

# --- Step 6: Run analysis for remaining tiles ---
for tile_path in tile_files:
    tile_id = os.path.basename(tile_path).replace("_timeseries.csv", "")
    print(f"\n🌾 Analyzing {tile_id}...")
    try:
        df_tile = pd.read_csv(tile_path)
        if not df_tile.empty:
            analyze_crop_health(df_tile, tile_id)
    except Exception as e:
        print(f"⚠️ Error processing {tile_id}: {e}")

# --- Step 7: Combine all summaries ---
print("\n📦 Combining all tile analyses...")
summary_files = glob.glob(f"{data_folder}/*_analysis.csv")
if summary_files:
    all_summaries = pd.concat([pd.read_csv(f) for f in summary_files], ignore_index=True)
    all_summaries.to_csv(f"{data_folder}/crop_health_summary.csv", index=False)
    print(f"✅ Combined summary saved: crop_health_summary.csv")
    display(all_summaries.head())
else:
    print("⚠️ No analysis summaries found yet.")


✅ Found 64 processed tiles. Resuming from remaining ones...
🧩 0 tiles left to process.

📦 Combining all tile analyses...
✅ Combined summary saved: crop_health_summary.csv


,tile_id,avg_health_index,avg_ndvi,avg_evi,avg_lst,avg_precip,avg_soil_moisture
0,tile_4_1,0.293476,4757.164261,2506.529220,15147.863265,2.735774,49.657052
1,tile_7_7,0.515256,5293.910574,3095.756996,15068.161369,2.399568,40.530958
2,tile_0_5,0.385042,4371.620642,2230.047924,15086.671114,3.040015,48.250277
3,tile_3_2,0.346582,4728.294970,2539.735430,15122.162574,2.559295,50.950119
4,tile_4_4,0.390328,4096.993123,2230.579237,15124.816787,2.537482,56.029553


In [35]:
data_folder = "tiles_data"
os.makedirs(data_folder, exist_ok=True)

def analyze_crop_health(df, tile_id):
    df = df.copy()
    df['ndvi_norm'] = (df['ndvi']-df['ndvi'].min())/(df['ndvi'].max()-df['ndvi'].min())
    df['evi_norm'] = (df['evi']-df['evi'].min())/(df['evi'].max()-df['evi'].min())
    df['lst_norm'] = (df['lst_day']-df['lst_day'].min())/(df['lst_day'].max()-df['lst_day'].min())
    df['moisture_score'] = (df['soil_moisture_surface'] + df['soil_moisture_root'])/2
    df['health_index'] = 0.4*df['ndvi_norm'] + 0.4*df['evi_norm'] - 0.2*df['lst_norm']

    summary = {
        "tile_id": tile_id,
        "avg_health_index": df['health_index'].mean(),
        "avg_ndvi": df['ndvi'].mean(),
        "avg_evi": df['evi'].mean(),
        "avg_lst": df['lst_day'].mean(),
        "avg_precip": df['precip'].mean(),
        "avg_soil_moisture": df['moisture_score'].mean()
    }

    pd.DataFrame([summary]).to_csv(f"{data_folder}/{tile_id}_analysis.csv", index=False)
    return summary


In [36]:
summary_files = []
for tile_path in glob.glob(f"{data_folder}/tile_*_timeseries.csv"):
    tile_id = tile_path.split('/')[-1].replace('_timeseries.csv','')
    df_tile = pd.read_csv(tile_path)
    if not df_tile.empty:
        analyze_crop_health(df_tile, tile_id)
summary_files = glob.glob(f"{data_folder}/tile_*_analysis.csv")
all_summaries = pd.concat([pd.read_csv(f) for f in summary_files], ignore_index=True)
all_summaries.to_csv(f"{data_folder}/crop_health_summary.csv", index=False)
print("✅ Crop health summary saved.")
all_summaries.head()


✅ Crop health summary saved.


,tile_id,avg_health_index,avg_ndvi,avg_evi,avg_lst,avg_precip,avg_soil_moisture
0,tile_4_1,0.293476,4757.164261,2506.529220,15147.863265,2.735774,49.657052
1,tile_7_7,0.515256,5293.910574,3095.756996,15068.161369,2.399568,40.530958
2,tile_0_5,0.385042,4371.620642,2230.047924,15086.671114,3.040015,48.250277
3,tile_3_2,0.346582,4728.294970,2539.735430,15122.162574,2.559295,50.950119
4,tile_4_4,0.390328,4096.993123,2230.579237,15124.816787,2.537482,56.029553


In [37]:
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv(f"{data_folder}/crop_health_summary.csv")
numeric_cols = data.select_dtypes(include=[np.number]).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(data[numeric_cols])
X_scaled = pd.DataFrame(X_scaled, columns=numeric_cols)
X_scaled['tile_id'] = data['tile_id']
print("✅ Normalized dataset ready for modeling.")
X_scaled.head()


✅ Normalized dataset ready for modeling.


,avg_health_index,avg_ndvi,avg_evi,avg_lst,avg_precip,avg_soil_moisture,tile_id
0,0.165423,0.597555,0.504247,0.767653,0.394729,0.588833,tile_4_1
1,0.677679,0.753270,0.766144,0.471687,0.191211,0.000000,tile_7_7
2,0.376918,0.485705,0.381358,0.540421,0.578898,0.498066,tile_0_5
3,0.288083,0.589180,0.519006,0.672216,0.287900,0.672265,tile_3_2
4,0.389127,0.406033,0.381594,0.682072,0.274696,1.000000,tile_4_4


In [38]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from imblearn.combine import SMOTETomek
import numpy as np
import joblib

# --- Features from scaled_df (all 5) ---
features = ['avg_ndvi', 'avg_evi', 'avg_lst', 'avg_precip', 'avg_soil_moisture']
X = scaled_df[features].values

# --- SMOTE + Tomek for cleaner balancing ---
smt = SMOTETomek(random_state=42)
X_bloom_res, y_bloom_res = smt.fit_resample(X, y_bloom_encoded)
X_crop_res, y_crop_res = smt.fit_resample(X, y_crop_encoded)

# --- Feature selection based on importance (optional, keeps all 5 features) ---
selector_bloom = SelectFromModel(RandomForestClassifier(n_estimators=300, random_state=42), threshold='median')
X_bloom_res_sel = selector_bloom.fit_transform(X_bloom_res, y_bloom_res)

selector_crop = SelectFromModel(RandomForestClassifier(n_estimators=300, random_state=42), threshold='median')
X_crop_res_sel = selector_crop.fit_transform(X_crop_res, y_crop_res)

# --- Pipelines ---
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('power', PowerTransformer(method='yeo-johnson')),
    ('rf', RandomForestClassifier(n_estimators=1000, max_depth=12, class_weight='balanced', n_jobs=-1, random_state=42))
])

xgb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('power', PowerTransformer(method='yeo-johnson')),
    ('xgb', XGBClassifier(
        n_estimators=800, learning_rate=0.03, max_depth=6, subsample=0.9, colsample_bytree=0.9,
        use_label_encoder=False, eval_metric='mlogloss', random_state=42, n_jobs=-1
    ))
])

et_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('power', PowerTransformer(method='yeo-johnson')),
    ('et', ExtraTreesClassifier(n_estimators=800, max_depth=None, class_weight='balanced', n_jobs=-1, random_state=42))
])

# --- Voting ensembles ---
bloom_ensemble = VotingClassifier(
    estimators=[('rf', rf_pipeline), ('xgb', xgb_pipeline), ('et', et_pipeline)],
    voting='soft',
    n_jobs=-1
)

crop_ensemble = VotingClassifier(
    estimators=[('rf', rf_pipeline), ('xgb', xgb_pipeline), ('et', et_pipeline)],
    voting='soft',
    n_jobs=-1
)

# --- CV evaluation ---
kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

print("🚀 Bloom Stage Cross-Validation...")
bloom_scores = cross_val_score(bloom_ensemble, X_bloom_res_sel, y_bloom_res, cv=kf, scoring='accuracy', n_jobs=-1)
print(f"✅ Bloom Accuracy (CV mean ± std): {bloom_scores.mean():.4f} ± {bloom_scores.std():.4f}")

print("\n🚀 Crop Recommendation Cross-Validation...")
crop_scores = cross_val_score(crop_ensemble, X_crop_res_sel, y_crop_res, cv=kf, scoring='accuracy', n_jobs=-1)
print(f"✅ Crop Accuracy (CV mean ± std): {crop_scores.mean():.4f} ± {crop_scores.std():.4f}")

# --- Fit final models ---
bloom_ensemble.fit(X_bloom_res_sel, y_bloom_res)
crop_ensemble.fit(X_crop_res_sel, y_crop_res)

# --- Save models ---
joblib.dump(bloom_ensemble, "tiles_data/best_bloom_stacked_full5.pkl")
joblib.dump(crop_ensemble, "tiles_data/best_crop_stacked_full5.pkl")

print("\n🎯 Models trained with all 5 features and saved successfully!")


🚀 Bloom Stage Cross-Validation...
✅ Bloom Accuracy (CV mean ± std): 0.9733 ± 0.0455

🚀 Crop Recommendation Cross-Validation...
✅ Crop Accuracy (CV mean ± std): 0.9735 ± 0.0573

🎯 Models trained with all 5 features and saved successfully!


In [39]:
# --- Ensure scaled_df exists and is in the expected form ---
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Path to the summary CSV you saved earlier
summary_csv = "tiles_data/crop_health_summary.csv"

# 1) Load file (will raise helpful error if missing)
try:
    scaled_df = pd.read_csv(summary_csv)
    print(f"✅ Loaded {summary_csv} with {len(scaled_df)} rows.")
except FileNotFoundError:
    raise FileNotFoundError(f"{summary_csv} not found. Make sure previous analysis produced tile_*_analysis.csv files and combined them into {summary_csv}.")

# 2) Ensure required feature columns exist (these names are used later)
expected_cols = ['avg_ndvi', 'avg_evi', 'avg_lst', 'avg_precip', 'avg_soil_moisture', 'tile_id']
missing = [c for c in expected_cols if c not in scaled_df.columns]
if missing:
    raise ValueError(f"Missing expected columns in {summary_csv}: {missing}. Check your analyze_crop_health() summary keys.")

# 3) Clean / fill NaNs
scaled_df[expected_cols[:-1]] = scaled_df[expected_cols[:-1]].astype(float)
scaled_df[expected_cols[:-1]] = scaled_df[expected_cols[:-1]].fillna(scaled_df[expected_cols[:-1]].mean())

# 4) If you already have true labels (bloom stage, crop type), load/prepare them here.
#    If not, create example labels to allow model pipeline to run. Replace this with your real labels.
if 'bloom_stage' in scaled_df.columns:
    y_bloom = scaled_df['bloom_stage']
else:
    # Example synthetic label: bloom if avg_ndvi is above median and health high — adjust as needed
    med_ndvi = scaled_df['avg_ndvi'].median()
    scaled_df['bloom_stage'] = (scaled_df['avg_ndvi'] > med_ndvi).astype(int)
    y_bloom = scaled_df['bloom_stage']
    print("⚠️ bloom_stage not found in CSV — created synthetic bloom_stage based on avg_ndvi median (replace with real labels).")

if 'recommended_crop' in scaled_df.columns:
    y_crop = scaled_df['recommended_crop']
else:
    # Example synthetic crop label: bucket avg_precip into 3 groups (0,1,2). Replace with your real crop labels.
    scaled_df['recommended_crop'] = pd.qcut(scaled_df['avg_precip'].rank(method='first'), q=3, labels=False)
    y_crop = scaled_df['recommended_crop']
    print("⚠️ recommended_crop not found — created synthetic recommended_crop (3 quantile groups). Replace with real crop labels.")

# 5) Encode targets if they are categorical
le_bloom = LabelEncoder()
y_bloom_encoded = le_bloom.fit_transform(y_bloom)

le_crop = LabelEncoder()
y_crop_encoded = le_crop.fit_transform(y_crop)

print(f"✅ Prepared labels: bloom classes={np.unique(y_bloom_encoded)}, crop classes={np.unique(y_crop_encoded)}")

# 6) Optionally create a scaled version (MinMax) named scaled_df to match later code usage.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
feat_cols = ['avg_ndvi', 'avg_evi', 'avg_lst', 'avg_precip', 'avg_soil_moisture']

scaled_values = scaler.fit_transform(scaled_df[feat_cols].values)
scaled_df_scaled = pd.DataFrame(scaled_values, columns=feat_cols)
scaled_df_scaled['tile_id'] = scaled_df['tile_id'].values
# If you want the variable name exactly 'scaled_df' (used later), set it:
scaled_df = scaled_df_scaled
print("✅ scaled_df created (normalized features). Sample:")
display(scaled_df.head())

# 7) Now you can safely run the block that defines `features = ['avg_ndvi', ...]` and uses scaled_df


✅ Loaded tiles_data/crop_health_summary.csv with 64 rows.
⚠️ bloom_stage not found in CSV — created synthetic bloom_stage based on avg_ndvi median (replace with real labels).
⚠️ recommended_crop not found — created synthetic recommended_crop (3 quantile groups). Replace with real crop labels.
✅ Prepared labels: bloom classes=[0 1], crop classes=[0 1 2]
✅ scaled_df created (normalized features). Sample:


,avg_ndvi,avg_evi,avg_lst,avg_precip,avg_soil_moisture,tile_id
0,0.597555,0.504247,0.767653,0.394729,0.588833,tile_4_1
1,0.753270,0.766144,0.471687,0.191211,0.000000,tile_7_7
2,0.485705,0.381358,0.540421,0.578898,0.498066,tile_0_5
3,0.589180,0.519006,0.672216,0.287900,0.672265,tile_3_2
4,0.406033,0.381594,0.682072,0.274696,1.000000,tile_4_4


In [40]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

# --- Generate future dates for 6 months (180 days) ---
future_dates = [date.today() + timedelta(days=i) for i in range(180)]

# --- Prepare future dataset with trend + small random variations ---
tiles = scaled_df['tile_id'].unique()
future_data = []

for tile in tiles:
    tile_row = scaled_df[scaled_df['tile_id'] == tile].iloc[0]

    # Random trend per feature
    ndvi_trend = np.random.uniform(-0.0005, 0.0005)
    evi_trend = np.random.uniform(-0.0005, 0.0005)
    lst_trend = np.random.uniform(-0.0002, 0.0002)
    precip_trend = np.random.uniform(-0.001, 0.001)
    moisture_trend = np.random.uniform(-0.001, 0.001)

    for i, d in enumerate(future_dates):
        future_data.append({
            'tile_id': tile,
            'date': d,
            'avg_ndvi': tile_row['avg_ndvi'] * (1 + ndvi_trend*i + np.random.normal(0, 0.02)),
            'avg_evi': tile_row['avg_evi'] * (1 + evi_trend*i + np.random.normal(0, 0.02)),
            'avg_lst': tile_row['avg_lst'] * (1 + lst_trend*i + np.random.normal(0, 0.01)),
            'avg_precip': tile_row['avg_precip'] * (1 + precip_trend*i + np.random.normal(0, 0.03)),
            'avg_soil_moisture': tile_row['avg_soil_moisture'] * (1 + moisture_trend*i + np.random.normal(0, 0.03))
        })

future_df = pd.DataFrame(future_data)
print(f"✅ Future prediction dataset created with {len(future_df)} rows")

# --- Define feature columns per model ---
bloom_features = ['avg_ndvi', 'avg_evi', 'avg_lst']             # replace with your bloom model features
crop_features = ['avg_ndvi', 'avg_precip', 'avg_soil_moisture']  # replace with your crop model features

# --- Prepare feature arrays ---
X_future_bloom = future_df[bloom_features].values
X_future_crop = future_df[crop_features].values

# --- Predict Bloom Stage using probability & custom threshold ---
bloom_probs = bloom_ensemble.predict_proba(X_future_bloom)[:, 1]
bloom_threshold = 0.52  # adjust threshold for realistic 0/1 mix
future_df['predicted_bloom_stage'] = (bloom_probs > bloom_threshold).astype(int)

# --- Predict Crop Recommendation ---
future_df['predicted_crop'] = crop_ensemble.predict(X_future_crop)

# --- Save CSV for frontend ---
future_df.to_csv("future_predictions_6months_dynamic_final_v3.csv", index=False)
print("✅ CSV generated: 'future_predictions_6months_dynamic_final_v3.csv'")

# --- Show sample predictions ---
future_df.head(15)


✅ Future prediction dataset created with 11520 rows
✅ CSV generated: 'future_predictions_6months_dynamic_final_v3.csv'


,tile_id,date,avg_ndvi,avg_evi,avg_lst,avg_precip,avg_soil_moisture,predicted_bloom_stage,predicted_crop
0,tile_4_1,2025-11-14,0.582245,0.483302,0.758763,0.396050,0.593279,1,2
1,tile_4_1,2025-11-15,0.610561,0.491412,0.756376,0.405899,0.588175,1,2
2,tile_4_1,2025-11-16,0.586010,0.507402,0.770867,0.393626,0.597800,1,2
3,tile_4_1,2025-11-17,0.580477,0.523895,0.758114,0.407834,0.610940,1,2
4,tile_4_1,2025-11-18,0.599305,0.509799,0.773138,0.405209,0.574544,1,2
5,tile_4_1,2025-11-19,0.587377,0.511359,0.773680,0.408283,0.602057,1,2
6,tile_4_1,2025-11-20,0.601408,0.502350,0.767391,0.422587,0.598365,1,2
7,tile_4_1,2025-11-21,0.570265,0.522954,0.753374,0.405522,0.597759,1,2
8,tile_4_1,2025-11-22,0.591343,0.496376,0.769644,0.405878,0.567880,1,2
9,tile_4_1,2025-11-23,0.600068,0.521545,0.755063,0.401182,0.589888,1,2


In [41]:
future_df.to_csv("future_predictions_6months.csv", index=False)
print("✅ CSV saved: future_predictions_6months.csv")


✅ CSV saved: future_predictions_6months.csv


In [42]:
!pip install folium

import folium
import pandas as pd
import numpy as np
# --- Load your predictions CSV ---
future_df = pd.read_csv("future_predictions_6months_dynamic_final_v3.csv")

# --- Define approximate coordinates for all 64 tiles ---
# Example: tiles evenly distributed in Nashik region (replace with actual lat/lon if available)
tiles = future_df['tile_id'].unique()
n_rows, n_cols = 8, 8  # 64 tiles arranged 8x8
lat_start, lat_end = 19.95, 20.10
lon_start, lon_end = 73.75, 73.85
tile_coords = {}

lats = np.linspace(lat_start, lat_end, n_rows)
lons = np.linspace(lon_start, lon_end, n_cols)

for i, tile in enumerate(tiles):
    r = i // n_cols
    c = i % n_cols
    tile_coords[tile] = (lats[r], lons[c])

# --- Map tile_id to coordinates ---
future_df['lat'] = future_df['tile_id'].map(lambda x: tile_coords.get(x, (0,0))[0])
future_df['lon'] = future_df['tile_id'].map(lambda x: tile_coords.get(x, (0,0))[1])

# --- Filter sample date for visualization ---
sample_date = "2025-11-15"
sample_df = future_df[future_df['date'] == sample_date]

# --- Create Folium map centered on Nashik ---
m = folium.Map(location=[20.0, 73.78], zoom_start=12)

# --- Add tiles as circle markers ---
for _, row in sample_df.iterrows():
    color = 'green' if row['predicted_bloom_stage'] == 1 else 'red'
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=8,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Tile: {row['tile_id']}\nBloom Stage: {row['predicted_bloom_stage']}\nCrop: {row['predicted_crop']}"
    ).add_to(m)

# --- Display the map ---
m


In [43]:
# --- Generate CSV with lat, lon included ---

import csv

output_csv_with_latlon = "/content/bloom_tiles_with_latlon.csv"

# Prepare header including latitude & longitude
fields = [
    "tile_id", "lat", "lon", "date",
    "avg_ndvi", "avg_evi", "avg_lst",
    "avg_precip", "avg_soil_moisture",
    "predicted_bloom_stage", "predicted_crop"
]

rows = []

for tile_info in tile_coords::
    tile_id = tile_info["tile_id"]
    avg_ndvi = tile_info.get("avg_ndvi", None)
    avg_evi = tile_info.get("avg_evi", None)
    avg_lst = tile_info.get("avg_lst", None)
    avg_precip = tile_info.get("avg_precip", None)
    avg_soil = tile_info.get("avg_soil_moisture", None)
    bloom_stage = tile_info.get("predicted_bloom_stage", None)
    crop_type = tile_info.get("predicted_crop", None)
    date = tile_info.get("date", None)

    # Extract lat, lon from tile geometry
    geom = tile_info.get("geometry")
    if geom:
        coords = geom.coordinates()[0][0]   # polygon → first ring → first vertex
        lon = coords[0]
        lat = coords[1]
    else:
        lat, lon = None, None

    rows.append([
        tile_id,
        lat,
        lon,
        date,
        avg_ndvi,
        avg_evi,
        avg_lst,
        avg_precip,
        avg_soil,
        bloom_stage,
        crop_type
    ])

# Write the new CSV
with open(output_csv_with_latlon, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    writer.writerows(rows)

print(f"CSV generated successfully: {output_csv_with_latlon}")


NameError: name 'tile_data_list' is not defined

In [32]:
[v for v in globals().keys() if "tile" in v.lower()]


['TILE_SIZE_DEG',
 'create_tiles',
 'tiles_gdf',
 'extract_features_for_tile',
 'all_tiles',
 'df_tile',
 'processed_tiles',
 'tile_files',
 'tile_path',
 'tile_id',
 'tiles',
 'tile',
 'tile_row',
 'tile_coords']

In [55]:
future_df

,tile_id,date,avg_ndvi,avg_evi,avg_lst,avg_precip,avg_soil_moisture,predicted_bloom_stage,predicted_crop,lat,lon
0,tile_4_1,2025-11-14,0.582245,0.483302,0.758763,0.396050,0.593279,1,2,19.95,73.75
1,tile_4_1,2025-11-15,0.610561,0.491412,0.756376,0.405899,0.588175,1,2,19.95,73.75
2,tile_4_1,2025-11-16,0.586010,0.507402,0.770867,0.393626,0.597800,1,2,19.95,73.75
3,tile_4_1,2025-11-17,0.580477,0.523895,0.758114,0.407834,0.610940,1,2,19.95,73.75
4,tile_4_1,2025-11-18,0.599305,0.509799,0.773138,0.405209,0.574544,1,2,19.95,73.75
...,...,...,...,...,...,...,...,...,...,...,...
11515,tile_0_7,2026-05-08,0.440599,0.388001,0.713615,0.448612,0.433150,0,2,20.10,73.85
11516,tile_0_7,2026-05-09,0.420267,0.382686,0.690463,0.459136,0.492235,0,2,20.10,73.85
11517,tile_0_7,2026-05-10,0.428444,0.365517,0.714762,0.476118,0.480467,0,2,20.10,73.85
11518,tile_0_7,2026-05-11,0.430679,0.378913,0.710115,0.470548,0.481837,0,2,20.10,73.85


In [56]:
future_df.to_csv("test.csv", index=False)
print("✅ CSV saved: test.csv")


✅ CSV saved: test.csv
